# Searching methods

So far, we have covered _dense_ vector search over documents, but searching documents has been around for quite a long time. Google has been good at this for decades - long before the rise in transformer architectures.

In this lesson we introduce **BM25**, a popular search algorithm, and one that you might hear being used in _hybrid search_ systems.

## TF-IDF
I just mentioned BM25, but before we get to that, let's talk about **TF-IDF**. This is a classic search algorithm that has been around for a long time. It stands for _Term Frequency-Inverse Document Frequency_.

TF-IDF is a fundamental statistical measure used in information retrieval and text mining to evaluate how important a word is to a document within a collection (corpus). It combines two key components:

- Term Frequency (TF): How often a word appears in a document
- Inverse Document Frequency (IDF): How unique or rare that word is across all documents

Before TF-IDF, systems often used simple word counts or frequencies, which had significant limitations:

- Common words (like "the", "is", "of") would dominate the results
- Rare but meaningful terms weren't given enough importance
- Document length differences weren't properly accounted for

Term Frequency (TF)
There are several ways to calculate TF:

Raw Frequency:
```text
tf(t,d) = number of times term t appears in document d
```

Boolean Frequency:
```text
tf(t,d) = 1 if t appears in d, 0 otherwise
```

Logarithmically Scaled Frequency:
```text
tf(t,d) = 1 + log(raw_frequency) if raw_frequency > 0, 0 otherwise
```

Normalized Frequency:
```text
tf(t,d) = raw_frequency / total_words_in_document
```

Inverse Document Frequency (IDF)
The IDF is usually given as
```text
idf(t) = log(N / (1 + df(t)))
```
where:
- `N` is the total number of documents in the corpus
- `df(t)` is the number of documents containing term `t`

You might see some different variations of the formula, but the idea is the same: the more documents that contain a term, the less unique or important it is.

The final calculation for TF-IDF is simply the product of TF and IDF:
```text
tfidf(t,d) = tf(t,d) * idf(t)
```

In [1]:
sentences = [
    "The quick brown fox jumps over the lazy dog",
    "A bright sunny day in the park",
    "Machine learning is a subset of artificial intelligence",
    "Dogs and cats are common pets",
    "The sun sets in the west",
    "Artificial intelligence is changing the world",
    "I love walking in the park on sunny days",
    "The lazy cat sleeps all day",
    "The trees in the park are beautiful",
]

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
vectorizer = TfidfVectorizer(stop_words='english')
sentence_vectors = vectorizer.fit_transform(sentences)

What is this `sentence_vectors` variable?

In [4]:
feature_names = vectorizer.get_feature_names_out()
dense_matrix = sentence_vectors.todense()

In [ ]:
feature_names

These are the words that appear within the documents, minus any stop words.

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame(
    dense_matrix,
    columns=feature_names,
    index=[f"Sentence {i+1}" for i in range(len(sentences))]
)

In [ ]:
# Print shape information
print("Matrix Shape:", sentence_vectors.shape)
print("Number of non-zero elements:", sentence_vectors.nnz)
print("\nVocabulary (feature names):", len(feature_names))
print(feature_names)

# Show non-zero elements in first sentence
print("\nNon-zero elements in first sentence:")
first_sentence = df.iloc[0]
print(first_sentence[first_sentence > 0].to_dict())

So each row of the matrix represents a document, and each column represents a word feature. The entry within each cell is the TF-IDF score for that word in that document.

Each row represents a sentence
```text
Row 1 = "The quick brown fox jumps over the lazy dog"
Row 2 = "A bright sunny day in the park"
...etc
```

Each column represents a unique word (after removing stop words)
```text
Column 1 = 'artificial'
Column 2 = 'bright'
Column 3 = 'brown'
...etc
```

For each word in each sentence
```text
0 = word doesn't appear
>0 = TF-IDF score for that word in that sentence
    - Higher if word appears frequently in sentence
    - Higher if word is rare across all sentences
```

Here is an example of how a word score varies across some sentences:

In [9]:
word = 'bright'
if word in feature_names:
    print(f"\nTF-IDF scores for word '{word}' across sentences:")
    for i, score in enumerate(df[word]):
        if score > 0:
            print(f"Sentence {i+1}: {score:.3f}")


TF-IDF scores for word 'bright' across sentences:
Sentence 2: 0.581


Bright only appears in the second sentence, so it has a high score there.

In [ ]:
word = 'park'
if word in feature_names:
    print(f"\nTF-IDF scores for word '{word}' across sentences:")
    for i, score in enumerate(df[word]):
        if score > 0:
            print(f"Sentence {i+1}: {score:.3f}")

Since "park" appears in many different sentences, it is not as unique, so it has a lower score.

The TF-IDF score for "park" can differ between sentences even if it appears once in each because of how the sentences are structured. Here's why:

1. Term Frequency (TF) part:

    - If one sentence is shorter, the same single occurrence of "park" would have a higher TF score because TF is typically normalized by document length

    - For example:

        - "Walk in the park" (park is 1/4 of terms)
        - "I love walking in the beautiful park near my house" (park is 1/9 of terms)


2. Context words matter:

    - If a sentence has more common words, the relative importance of "park" increases
    
    - If a sentence has more rare/unique words, the relative weight of each term is distributed differently


3. ormalization:

    - TF-IDF vectors are usually normalized (like in sklearn)

    - So if a sentence contains other high-scoring terms, it will reduce the final score for "park" in that sentence due to the normalization

    - For example, if "park" appears with very rare words, its relative contribution to the normalized vector will be smaller than if it appears with common words

Even though "park" might appear once in multiple sentences, its final TF-IDF score can vary significantly based on sentence length, the other words present, and normalization effects.

### Searcing with TF-IDF
OK, great, but how do we search with TF-IDF?

In [11]:
query = "bright sunny park hippo"
n_results = 5
# Transform the query into a vector
query_vector = vectorizer.transform([query])


What is actually going on during the transformation process? For an example query as above, what is the output of the `transform` method?

In [ ]:
query_vector.todense()

We only have 3 words in the query that are in the vocabulary. The word 'hippo' is not in the vocabulary, so it is ignored.

If we look at the source code, we see that actually, we first apply the `CountVectorizer` to get the TF, then we simply use the IDF values that we found during the `fit` step.

In [13]:
similarities = cosine_similarity(query_vector, sentence_vectors).flatten()
top_indices = similarities.argsort()[::-1][:n_results]

In [14]:
results = []
for idx in top_indices:
    if similarities[idx] > 0:
        results.append({
            'score': round(similarities[idx], 3),
            'text': sentences[idx]
        })

In [ ]:
results

Notice that for the top result, the words "bright", "sunny" and "park" are all present in the query. This is why the score is so high.

In contrast, the second result has "bright" and "sunny", but not "park", so the score is lower. And the final result has only the word "park", so the score is even lower.

## A real world example
OK, let's use this in anger.

We have a dataset of scraped documentation from the Instructor python Library. We want to search this dataset for relevant documents based on a query.

First we gather all of the above into a convenient class.

In [16]:
import os
from pathlib import Path
from typing import List, Union
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from llama_index.core import SimpleDirectoryReader

class TFIDFSearch:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.doc_vectors = None
        self.doc_paths = []
    
    
    def read_file(self, file_path):
        """Read content from a file."""
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()

    def load_documents(self, directory: Union[str, Path]) -> List[str]:
        files = SimpleDirectoryReader(directory, recursive=True).load_data()
        cwd = os.getcwd()
        
        # Create dictionary to store unique files
        unique_files = {}
        for doc in files:
            path = "."+doc.metadata['file_path'][len(cwd):]
            if path not in unique_files:
                unique_files[path] = doc.text
        
        # Update class attributes
        self.doc_paths = list(unique_files.keys())
        return list(unique_files.values())

    def fit_from_directory(self, directory: Union[str, Path]):
        """
        Load documents from a directory and fit the TF-IDF model.
        
        Args:
            directory: Path to directory containing text files
        """
        # Load documents
        documents = self.load_documents(directory)
        
        # Fit vectorizer and transform documents
        self.doc_vectors = self.vectorizer.fit_transform(documents)
        
        print(f"Processed {len(documents)} documents")
        print(f"Vocabulary size: {len(self.vectorizer.get_feature_names_out())}")

    def search(self, query: str, n: int = 5) -> list[tuple]:
        """
        Search for documents matching the query text.
        
        Args:
            query: Raw query string
            n: Number of documents to return
            
        Returns:
            List of (score, file_path) tuples
        """
        query_vector = self.vectorizer.transform([query])
        similarities = cosine_similarity(query_vector, self.doc_vectors).flatten()
        top_indices = similarities.argsort()[::-1][:n]
        
        results = []
        for idx in top_indices:
            if similarities[idx] > 0:  # Only include non-zero scores
                text = self.read_file(self.doc_paths[idx])
                results.append((similarities[idx], self.doc_paths[idx], text))
        
        return results

    def get_feature_names(self):
        """Return the feature names (vocabulary)."""
        return self.vectorizer.get_feature_names_out()



In [ ]:
searcher = TFIDFSearch()

searcher.fit_from_directory("data/test")

In [ ]:
query = "bright sunny park hippo"
results = searcher.search(query)

print("\nSearch results:")
for score, path, text in results:
    print(f"Score: {score:.4f}, File: {path}, Text: {text}")

Great so this works as expected. We can search for documents based on a query. Now let's apply it to the Instructor documentation.

In [ ]:
searcher = TFIDFSearch()
searcher.fit_from_directory("data/docs")

There are a bunch of errors. Don't worry about those, it's because we're trying to parse images. We'll fix that later.

In [ ]:
query = "I am trying to extract some data from a receipt using OpenAI models. Can you help me with that?"
results = searcher.search(query, n=10)

print("\nSearch results:\n")
for score, path, text in results:
    print(f"Score: {score:.4f}, File: {path}")

Well, that top one looks promising! What does it say?

# Extracting Receipt Data using GPT-4 and Python

This post demonstrates how to use Python's Pydantic library and OpenAI's GPT-4 model to extract receipt data from images and validate the total amount. This method is particularly useful for automating expense tracking and financial analysis tasks.

## Defining the Item and Receipt Classes

First, we define two Pydantic models, `Item` and `Receipt`, to structure the extracted data. The `Item` class represents individual items on the receipt, with fields for name, price, and quantity. The `Receipt` class contains a list of `Item` objects and the total amount.

```python
from pydantic import BaseModel


class Item(BaseModel):
    name: str
    price: float
    quantity: int


class Receipt(BaseModel):
    items: list[Item]
    total: float
```

## Validating the Total Amount

To ensure the accuracy of the extracted data, we use Pydantic's `model_validator` decorator to define a custom validation function, `check_total`. This function calculates the sum of item prices and compares it to the extracted total amount. If there's a discrepancy, it raises a `ValueError`.

```python
from pydantic import model_validator


@model_validator(mode="after")
def check_total(self):
    items = self.items
    total = self.total
    calculated_total = sum(item.price * item.quantity for item in items)
    if calculated_total != total:
        raise ValueError(
            f"Total {total} does not match the sum of item prices {calculated_total}"
        )
    return self
```

## Extracting Receipt Data from Images

The `extract_receipt` function uses OpenAI's GPT-4 model to process an image URL and extract receipt data. We utilize the `instructor` library to configure the OpenAI client for this purpose.

```python
import instructor
from openai import OpenAI

# <%hide%>
from pydantic import BaseModel, model_validator


class Item(BaseModel):
    name: str
    price: float
    quantity: int


class Receipt(BaseModel):
    items: list[Item]
    total: float

    @model_validator(mode="after")
    def check_total(cls, values: "Receipt"):
        items = values.items
        total = values.total
        calculated_total = sum(item.price * item.quantity for item in items)
        if calculated_total != total:
            raise ValueError(
                f"Total {total} does not match the sum of item prices {calculated_total}"
            )
        return values


# <%hide%>

client = instructor.from_openai(OpenAI())


def extract(url: str) -> Receipt:
    return client.chat.completions.create(
        model="gpt-4",
        max_tokens=4000,
        response_model=Receipt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": url},
                    },
                    {
                        "type": "text",
                        "text": "Analyze the image and return the items in the receipt and the total amount.",
                    },
                ],
            }
        ],
    )
```

## Practical Examples

In these examples, we apply the method to extract receipt data from two different images. The custom validation function ensures that the extracted total amount matches the sum of item prices.

```python
# <%hide%>
from pydantic import BaseModel, model_validator
import instructor
from openai import OpenAI


class Item(BaseModel):
    name: str
    price: float
    quantity: int


class Receipt(BaseModel):
    items: list[Item]
    total: float

    @model_validator(mode="after")
    def check_total(cls, values: "Receipt"):
        items = values.items
        total = values.total
        calculated_total = round(sum(item.price * item.quantity for item in items), 2)
        if calculated_total != total:
            raise ValueError(
                f"Total {total} does not match the sum of item prices {calculated_total}"
            )
        return values


client = instructor.from_openai(OpenAI())


def extract(url: str) -> Receipt:
    return client.chat.completions.create(
        model="gpt-4o",
        max_tokens=4000,
        response_model=Receipt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": url},
                    },
                    {
                        "type": "text",
                        "text": "Analyze the image and return the items in the receipt and the total amount.",
                    },
                ],
            }
        ],
    )


# <%hide%>
url = "https://templates.mediamodifier.com/645124ff36ed2f5227cbf871/supermarket-receipt-template.jpg"


receipt = extract(url)
print(receipt)
```

By combining the power of GPT-4 and Python's Pydantic library, we can accurately extract and validate receipt data from images, streamlining expense tracking and financial analysis tasks.


Let's try another example:

In [ ]:
query = "How do I use Instructor with OpenAI models?"
results = searcher.search(query, n=10)

print("\nSearch results:\n")
for score, path, text in results:
    print(f"Score: {score:.4f}, File: {path}")

Well this is not so good. None of these scores are particularly useful to me...

## BM25
There are a number of ways to improve on basic TF-IDF search. One popular method is **BM25**.

#### Term Saturation
If the word cat appears 10 times in a document, it's probably not 10 times more important than if it appeared once. BM25 accounts for this by using a saturation function.

```text
term_score = tf / (tf + k)
```

In addition, we also care about the document length. If a document contains the word "cat" only once and it is short, then the word "cat" is still likely to be relevant. However, if the document is very long, then the word "cat" is less likely to be relevant.

As the k parameter increases, the value of TF/(TF + k) decreases. To penalize long documents, we can increase k if the document is longer than average, and decrease k if the document is shorter than average. We’ll achieve this by multiplying k by the ratio dl/adl. Here, dl is the document’s length, and adl is the average document length across the corpus.


We're not going to implement BM25 from scratch, so we use a library instead.

In [26]:
import os
from pathlib import Path
from typing import List, Union
import bm25s
from llama_index.core import SimpleDirectoryReader

class BM25Search:
    def __init__(self):
        self.retriever = None
        self.doc_paths = []
        self.documents = []
    
    def read_file(self, file_path):
        """Read content from a file."""
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()

    def load_documents(self, directory: Union[str, Path]) -> List[str]:
        files = SimpleDirectoryReader(directory, recursive=True).load_data()
        cwd = os.getcwd()
        
        # Create dictionary to store unique files
        unique_files = {}
        for doc in files:
            path = "."+doc.metadata['file_path'][len(cwd):]
            if path not in unique_files:
                unique_files[path] = doc.text
        
        # Update class attributes
        self.doc_paths = list(unique_files.keys())
        return list(unique_files.values())

    def fit_from_directory(self, directory: Union[str, Path], **kwargs):
        """
        Load documents from a directory and create the BM25 index.
        
        Args:
            directory: Path to directory containing text files
        """
        # Load documents
        self.documents = self.load_documents(directory)
        
        # Tokenize documents and create index
        corpus_tokens = bm25s.tokenize(self.documents, stopwords="en")
        self.retriever = bm25s.BM25(**kwargs)
        self.retriever.index(corpus_tokens)
        
        print(f"Processed {len(self.documents)} documents")

    def search(self, query: str, n: int = 5) -> list[tuple]:
        """
        Search for documents matching the query text.
        
        Args:
            query: Raw query string
            n: Number of documents to return
            
        Returns:
            List of (score, file_path, text) tuples
        """
        query_tokens = bm25s.tokenize(query, stopwords="en")
        docs, scores = self.retriever.retrieve(query_tokens, corpus=self.documents, k=n)
        
        results = []
        # Make sure we're accessing the first element of docs and scores
        for doc, score in zip(docs[0], scores[0]):
            if score > 0:  # Only include non-zero scores
                doc_idx = self.documents.index(doc)
                text = self.read_file(self.doc_paths[doc_idx])
                results.append((score, self.doc_paths[doc_idx], text))
        
        return sorted(results, key=lambda x: x[0], reverse=True)

    def save_index(self, path: str):
        """Save the BM25 index to disk."""
        self.retriever.save(path)
    
    def load_index(self, path: str):
        """Load a previously saved BM25 index."""
        self.retriever = bm25s.BM25.load(path, load_corpus=True)

Not much else changes. We can test it out with the test sentences.

In [ ]:
searcher = BM25Search()
searcher.fit_from_directory("data/test")

results = searcher.search("bright sunny park hippo", n=3)
print("\nSearch results:")
for score, path, text in results:
    print(f"Score: {score:.4f}, File: {path}, Text: {text}")

Since that works, let's try with the Instructor documentation.

In [ ]:
searcher = BM25Search()
searcher.fit_from_directory("data/docs")

results = searcher.search("I am trying to extract some data from a receipt using OpenAI models.", n=10)
print("\nSearch results:")
for score, path, text in results:
    print(f"Score: {score:.4f}, File: {path}")

In [ ]:
results = searcher.search("How do I use Instructor with OpenAI models?", n=10)
print("\nSearch results:")
for score, path, text in results:
    print(f"Score: {score:.4f}, File: {path}")

Well, at least these aren't just blog posts... but still, not very useful. Things to play with include the `k1` and `b` parameters.

In [ ]:
searcher = BM25Search()
searcher.fit_from_directory("data/docs", k1=0.5,
        b=1.2,
        delta=0.1)

results = searcher.search("How do I use Instructor with OpenAI models?", n=10)
print("\nSearch results:")
for score, path, text in results:
    print(f"Score: {score:.4f}, File: {path}")

In the next section, we look at how well dense vector searches will do.